In [1]:
%reload_ext autoreload
%autoreload 2

import os
import glob
from pathlib import Path

import numpy as np
import pandas as pd

from tqdm import tqdm
from hbmep.utils import timing
import mat73
import tomllib


In [2]:
MUSCLES = ["ADM", "APB", "Biceps", "ECR", "FCR", "Triceps"]
PKPK_MUSCLES = ["PKPK_" + m for m in MUSCLES]
AUC_MUSCLES = ["AUC_" + m for m in MUSCLES]

@timing
def load_non_invasive_data(
    dir: Path,
    subjects: list[str]
):
    df = None

    for subject in tqdm(subjects):
        print(subject)
        subdir = os.path.join(dir, subject)

        fpath = glob.glob(f"{subdir}/*REC_table.csv")[0]
        temp_df = pd.read_csv(fpath)

        fpath = glob.glob(f"{subdir}/*ep_matrix.mat")[0]
        data_dict = mat73.loadmat(fpath)

        temp_mat = data_dict["ep_sliced"]

        fpath = glob.glob(f"{subdir}/*cfg_proc.toml")[0]
        with open(fpath, "rb") as f:
            cfg_proc = tomllib.load(f)

        temp_df["participant"] = subject

        # Rename columns to actual muscle names
        muscles = cfg_proc["st"]["channel"]
        pkpk_muscles_map = {
            f"pkpk_{i + 1}": "PKPK_" + m for i, m in enumerate(muscles)
        }
        auc_muscles_map = {
            f"auc_{i + 1}": "AUC_" + m for i, m in enumerate(muscles)
        }
        temp_df = temp_df.rename(columns=pkpk_muscles_map).copy()
        temp_df = temp_df.rename(columns=auc_muscles_map).copy()

        # Reorder MEP matrix
        temp_mat = temp_mat[..., np.argsort(muscles)]

        assert temp_df["target_muscle"].unique().shape[0] == 1
        side = temp_df["target_muscle"].unique()[0][0]

        pkpk_side_muscles = ["PKPK_" + side + muscle for muscle in MUSCLES]
        temp_df[PKPK_MUSCLES] = temp_df[pkpk_side_muscles]

        auc_side_muscles = ["AUC_" + side + muscle for muscle in MUSCLES]
        temp_df[AUC_MUSCLES] = temp_df[auc_side_muscles]

        side_muscles = [side + muscle for muscle in MUSCLES]

        ind = [i for i, m in enumerate(sorted(muscles)) if m in side_muscles]
        temp_mat = temp_mat[..., ind]

        if df is None:
            df = temp_df.copy()
            mat = temp_mat

            muscles_sorted = sorted(muscles)

            assert len(set(muscles_sorted)) == len(muscles_sorted)
            continue

        assert set(muscles) == set(muscles_sorted)

        df = pd.concat([df, temp_df], ignore_index=True).reset_index(drop=True).copy()
        mat = np.vstack((mat, temp_mat))

    return df, mat, pkpk_muscles_map, auc_muscles_map


In [3]:
dir = "/mount/hdd1/human_non-inv/proc_2023-11-28/tms-visit-last-most"

subjects = [
    "SCA01",
    "SCA02",
    "SCA03",
    "SCA04",
    "SCA05",
    "SCA06",
    "SCA07",
    "SCA09",
    "SCA10",
    "SCA11",
    "SCA13",
    "SCA14",
    "SCS01",
    "SCS02",
    "SCS03",
    "SCS04",
    "SCS05",
    "SCS06",
    "SCS08",
]

df, mat, pkpk_muscles_map, auc_muscles_map = load_non_invasive_data(dir=dir, subjects=subjects)
df.shape

  0%|                                                    | 0/19 [00:00<?, ?it/s]

SCA01


  5%|██▎                                         | 1/19 [00:00<00:03,  4.81it/s]

 11%|████▋                                       | 2/19 [00:00<00:03,  5.44it/s]

SCA02
SCA03


...

SCS04
SCS05


 95%|████████████████████████████████████████▋  | 18/19 [00:02<00:00,  6.98it/s]

SCS06
SCS08


100%|███████████████████████████████████████████| 19/19 [00:02<00:00,  6.47it/s]
2023-11-28 16:40:54,336 - hbmep.utils.utils - INFO - func:load_non_invasive_data took: 2.94 sec


(1415, 119)

In [4]:
mat.shape

(1415, 1501, 6)

In [5]:
target_df = df \
    .groupby(by=["participant"], as_index=False) \
    .agg({"target_muscle": np.unique}) \
    .explode(column="target_muscle") \
    .reset_index(drop=True) \
    .copy()

target_df

,participant,target_muscle
0,SCA01,LAPB
1,SCA02,RAPB
2,SCA03,RAPB
3,SCA04,RAPB
4,SCA05,RAPB
5,SCA06,RAPB
6,SCA07,RAPB
7,SCA09,RAPB
8,SCA10,RAPB
9,SCA11,RAPB


In [6]:
for subject in df.participant.unique():
    ind = df.participant.isin([subject])
    temp_df = df[ind].reset_index(drop=True).copy()

    side = temp_df.target_muscle.unique()[0][0]
    pkpk_sided_muscles = ["PKPK_" + side + m for m in MUSCLES]
    auc_sided_muscles = ["AUC_" + side + m for m in MUSCLES]

    assert (temp_df[PKPK_MUSCLES].to_numpy() == temp_df[pkpk_sided_muscles].to_numpy()).all()
    assert (temp_df[AUC_MUSCLES].to_numpy() == temp_df[auc_sided_muscles].to_numpy()).all()


In [7]:
dir = "/home/vishu/data/hbmep-processed/human/tms"

dest = os.path.join(dir, "proc_2023-11-28.csv")
df.to_csv(dest, index=False)
print("data", dest)

dest = os.path.join(dir, "proc_2023-11-28.npy")
np.save(dest, mat)
print("mat", dest)


data /home/vishu/data/hbmep-processed/human/tms/proc_2023-11-28.csv
mat /home/vishu/data/hbmep-processed/human/tms/proc_2023-11-28.npy


In [8]:
import json

dest = os.path.join(dir, "pkpk_muscles_map.json")
f = open(dest, "w")
f.write(json.dumps(pkpk_muscles_map))
f.close;

dest = os.path.join(dir, "auc_muscles_map.json")
f = open(dest, "w")
f.write(json.dumps(auc_muscles_map))
f.close;


In [9]:
pkpk_muscles_map


{'pkpk_1': 'PKPK_LBiceps',
 'pkpk_2': 'PKPK_LTriceps',
 'pkpk_3': 'PKPK_LFCR',
 'pkpk_4': 'PKPK_LECR',
 'pkpk_5': 'PKPK_LAPB',
 'pkpk_6': 'PKPK_LADM',
 'pkpk_7': 'PKPK_LFDI',
 'pkpk_8': 'PKPK_RBiceps',
 'pkpk_9': 'PKPK_RTriceps',
 'pkpk_10': 'PKPK_RFCR',
 'pkpk_11': 'PKPK_RECR',
 'pkpk_12': 'PKPK_RAPB',
 'pkpk_13': 'PKPK_RADM',
 'pkpk_14': 'PKPK_RFDI',
 'pkpk_15': 'PKPK_trigger_tscs',
 'pkpk_16': 'PKPK_trigger_peripheral',
 'pkpk_17': 'PKPK_trigger_tms'}

In [10]:
auc_muscles_map


{'auc_1': 'AUC_LBiceps',
 'auc_2': 'AUC_LTriceps',
 'auc_3': 'AUC_LFCR',
 'auc_4': 'AUC_LECR',
 'auc_5': 'AUC_LAPB',
 'auc_6': 'AUC_LADM',
 'auc_7': 'AUC_LFDI',
 'auc_8': 'AUC_RBiceps',
 'auc_9': 'AUC_RTriceps',
 'auc_10': 'AUC_RFCR',
 'auc_11': 'AUC_RECR',
 'auc_12': 'AUC_RAPB',
 'auc_13': 'AUC_RADM',
 'auc_14': 'AUC_RFDI',
 'auc_15': 'AUC_trigger_tscs',
 'auc_16': 'AUC_trigger_peripheral',
 'auc_17': 'AUC_trigger_tms'}

In [11]:
src = "/mount/hdd1/human_non-inv/proc-2023-11-20/tms-visit-last-most/SCA01/SCA01_V1T0_TMS_REC_ep_matrix.mat"
data_dict = mat73.loadmat(src)

t = data_dict["t_sliced"]
print(t.min(), t.max())


-0.15 0.15


In [12]:
df.shape


(1241, 119)

In [13]:
mat.shape


(1241, 1501, 6)

In [14]:
sorted(MUSCLES) == MUSCLES


True